In [1]:
import json
import os
from transformers import EncoderDecoderModel, BertTokenizer, BertModel, BertForNextSentencePrediction, BatchEncoding
import torch
from torch.nn import Softmax
from typing import List
import sys

sys.path.append('..')
from tools.TextProcessing import build_word_tree, process_keywords, batched_sent_tokenize, clean_text
from tools.BasicUtils import my_write, my_csv_read, MultiThreading, get_wikipedia_entity

In [ ]:
file_description = [
    "keyword_f.txt ---- CS keywords",
    "wordtree.json ---- word tree for cs keywords",
    "entity.txt ---- Reformed cs keywords with '_' replacing ' '"
]

if not os.path.exists('../data/temp/joint_score_function'):
    os.mkdir('../data/temp/joint_score_function')
    
my_write('../data/temp/joint_score_function/readme.txt', file_description)

## Prepare the data

In [ ]:
# Collect keywords from terms-cs-cfl-epoch200.txt
stable_kw = []
unstable_kw = []
r = my_csv_read('../data/raw_data/terms-cs-cfl-epoch200.txt', delimiter='\t')
candidate_kw_list = [item[0] for item in r if float(item[1]) > 0.1]
stable_kw, unstable_kw = process_keywords(candidate_kw_list)
# Save keywords
my_write('../data/temp/joint_score_function/keyword_f.txt', stable_kw)
# Generate word tree (25 seconds)
build_word_tree('../data/temp/joint_score_function/keyword_f.txt', '../data/temp/joint_score_function/wordtree.json', '../data/temp/joint_score_function/entity.txt')

In [ ]:
# Go to py folder and run followings in the backend 
# "python gen_co_occur.py ../data/temp/joint_score_function/wordtree.json ../data/corpus/small_sent.txt ../data/temp/joint_score_function/co_occur.txt"
# "python gen_occur.py ../data/temp/joint_score_function/keyword_f.txt ../data/temp/joint_score_function/co_occur.txt ../data/temp/joint_score_function/occur.json"

## Setup model

In [2]:
class ScoreFunction(torch.nn.Module):
    def __init__(self, model_file:str, additional_special_tokens:List[str]=None, device:str=None):
        super().__init__()
        self._score_function = BertForNextSentencePrediction.from_pretrained(model_file)
        self._tokenizer = BertTokenizer.from_pretrained(model_file)
        self._device = torch.device("cuda" if torch.cuda.is_available() else "cpu") if device is None else torch.device(device)
        self._sm = Softmax(1)
        
        if additional_special_tokens is not None:
            self._tokenizer.add_special_tokens({'additional_special_tokens' : additional_special_tokens})
            self._score_function.resize_token_embeddings(len(self._tokenizer))
        self._score_function.to(self._device)

    def forward(self, original_sents:List[str], queries:List[str]):
        inputs = BatchEncoding(self._tokenizer(original_sents, queries, padding=True, truncation=True, max_length=80, return_tensors="pt")).to(self._device)
        output = self._score_function(**inputs, labels=torch.LongTensor([1]*len(original_sents)).to(self._device))
        return self._sm(output.logits)[:, 1]

In [3]:
sf = ScoreFunction('bert-base-uncased', additional_special_tokens=['<RELATION>'])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
# initialize Bert2Bert from pre-trained checkpoints
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [10]:
bert2bert = EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-uncased', 'bert-base-uncased')
bert2bert.config.decoder_start_token_id = tokenizer.cls_token_id
bert2bert.config.eos_token_id = tokenizer.sep_token_id

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relations

## Training